In [66]:
import pandas as pd 
import numpy as np 
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import xgboost as xgb
from xgboost import XGBClassifier 
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score


In [12]:
dataset = pd.read_csv("train_final.csv")
test_dataset = pd.read_csv("test_final.csv")
test_dataset.head()

,Id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24
0,16384,37733,1,1.77,118603,1,0,118602,118097,1,...,13881,117941,117887,1,117885,1,1,1,1,1
1,16385,312129,1,3.54,118052,1,0,117961,290919,1,...,14638,118992,290919,1,118321,1,1,1,7,1
2,16386,24884,1,23.01,118300,1,0,117961,302830,1,...,770,119181,4673,1,128230,1,1,1,14,1
3,16387,4674,1,1.77,119091,1,0,119062,118036,1,...,16752,143531,290919,1,117905,1,1,1,81,1
4,16388,68725,1,3.54,118300,1,0,117961,171056,1,...,4945,118360,118638,1,118636,1,1,1,1,1


In [64]:
feature_cols = ['Id','f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16','f17','f18','f19','f20','f21','f22','f23','f24']
features_X = dataset[feature_cols]
target_Y = dataset['Y']
features_X.head()

,Id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24
0,1,25884,1,33.63,118596,1,0,118595,125738,1,...,1945,118450,119184,1,121372,1,1,1,2,1
1,2,34346,1,10.62,118041,1,0,117902,130913,1,...,15385,117945,292795,1,259173,1,1,1,1,1
2,3,34923,1,1.77,118327,1,0,117961,124402,1,...,7547,118933,290919,1,118784,1,1,1,1,1
3,4,80926,1,30.09,118300,1,0,117961,301218,1,...,4933,118458,118331,1,307024,1,1,1,2,1
4,5,4674,1,1.77,119921,1,0,119920,302830,1,...,13836,142145,4673,1,128230,1,1,1,620,1


In [65]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Disbursed'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print "\nModel Report"
    print "Accuracy : %.4g" % metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions)
    print "AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob)
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("\nModel Report")? (<ipython-input-65-d3a582a439d9>, line 18)

In [61]:
X_train,X_test,y_train,y_test=train_test_split(features_X,target_Y,test_size=0.25,random_state=42)

# logreg = LogisticRegression()

# logreg.fit(X_train, y_train)

# y_pred = logreg.predict(X_test)

model = XGBClassifier(nthread = 8, eval_metric='logloss', eta = 0.1, max_depth=5, min_child_weight=1, gamma=0,subsample=0.8,cosample_bytree=0.8,scale_pos_weight=1)
model.fit(X_train, y_train)
print(model)

y_pred = model.predict_proba(X_test)[:,1]
# accuracy = accuracy_score(y_test, y_pred)

# print(accuracy)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='logloss', gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=8, objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)


In [62]:
submission = pd.DataFrame({'Id': test_dataset['Id'], 'Y':model.predict_proba(test_dataset)[:,1]})
submission.head()

,Id,Y
0,16384,0.957580
1,16385,0.765922
2,16386,0.997490
3,16387,0.996992
4,16388,0.969709


In [63]:
filename = 'submit.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: submit.csv
